In [1]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm

from model import SchellingModel
from mesa.batchrunner import BatchRunner

from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami
import numpy as np
import math

In [2]:
# Define the model inputs
n = 2
problem = {
    'num_vars': n,
    'names': ['mobil', 'var'],
    'bounds': [[0.01, 0.05],
               [0.005, 0.05]]
}

# Generate samples
param_values = saltelli.sample(problem, 10)

Y = np.zeros([param_values.shape[0]])

with tqdm(param_values.size, disable=False) as pbar:
    for i, X in enumerate(param_values):
        model = SchellingModel(25, 25, 0.0015, X[0], 0.50, X[1], 0)
        while model.running:
            model.step()
        Y[i] = model.gent_time
        pbar.update()


In [14]:
# remove infeasible solutions
for i in range(0,Y.size,2*n+2):
    if math.isnan(sum(Y[i:i+2*n+2])):
        Y = np.delete(Y,range(i,i+2*n+2))

In [4]:
# Perform analysis
Si = sobol.analyze(problem, Y, print_to_console=True)

# Print the first-order sensitivity indices
print(Si['S1'])

Parameter S1 S1_conf ST ST_conf
mobil -0.102040 0.246051 0.993761 0.363719
var 0.552783 0.301674 1.247006 0.483292

Parameter_1 Parameter_2 S2 S2_conf
mobil var -0.352434 0.411353
[-0.10204005  0.55278318]
